In [2]:
# Augmente la taille du notebook pour occuper toute la page
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
#Importation des modules utilisés
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nba_api
import sys
sys.path.append("/Users/rapha/Documents/DataScientest/Projet/JAN23_BDS_NBA") # Ajout du répertoire où se trouve nba_common_library
from nba_common_library import lib_string, utils

sns.set_theme()

In [3]:
# Chargement des jeux de données
# Note: les chemins des fichiers sont ceux en local sur mon pc
#       A modifier si nécessaire
shot_data = pd.read_csv("../Data/NBA Shot Locations 1997 - 2020.csv") # Données de chaque tir (distance, temps restant, période du match, etc.), contient la variable cible Shot Made Flag
player_data = pd.read_csv("../Data/Players.csv", index_col=0) # Données individuelles de chaque joueur (taille, poids, date et lieu de naissance, etc.)
pbp_data = pd.read_csv('../Data/pbp_concat.csv') # Données détaillées de chaque action (pas seulement les tirs) par match

In [11]:
# On enlève les 2 premières colonnes inutiles dans pbp_data
pbp_data.drop(pbp_data.columns[:2], axis=1, inplace=True)

In [21]:
# On récupère des noms et IDs uniques des joueurs avec nba_pi
from nba_api.stats.static import players
all_players = players.get_players()
player_id = [player_info['id'] for player_info in all_players]
player_first_name = [player_info['first_name'] for player_info in all_players]
player_last_name = [player_info['last_name'] for player_info in all_players]
player_full_name = [player_info['full_name'] for player_info in all_players]

# Fonction qui associe l'ID du joueur à parti de son nom et prénom
# si ils se trouvent dans la liste des joueurs connus
def name_to_player_id(full_name):
    if full_name in player_full_name:
        return player_id[player_full_name.index(full_name)]
    else:
        return np.nan
    
# On ajoute l'ID des joueurs à player_data
player_data['Player ID'] = player_data['Player'].astype('str').apply(name_to_player_id)
player_data = player_data.dropna(axis=0, subset=['Player ID'])
player_data['Player ID'] = player_data['Player ID'].astype(int)

C:\Users\rapha\AppData\Local\Temp\ipykernel_4040\1403243630.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['Player ID'] = player_data['Player ID'].astype(int)


In [38]:
# On ajoute les données individuelles de chaque joueur dans shot_data 
# en réalisant une fusion sur la colonne Player ID
shot_data = shot_data.merge(player_data, on='Player ID', how = 'inner')
shot_data.drop('Player', axis=1, inplace=True) # colonne redondante avec Player Name

In [40]:
display(shot_data.info())
display(pbp_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4341732 entries, 0 to 4341731
Data columns (total 28 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Game ID            int64  
 1   Game Event ID      int64  
 2   Player ID          int64  
 3   Player Name        object 
 4   Team ID            int64  
 5   Team Name          object 
 6   Period             int64  
 7   Minutes Remaining  int64  
 8   Seconds Remaining  int64  
 9   Action Type        object 
 10  Shot Type          object 
 11  Shot Zone Basic    object 
 12  Shot Zone Area     object 
 13  Shot Zone Range    object 
 14  Shot Distance      int64  
 15  X Location         int64  
 16  Y Location         int64  
 17  Shot Made Flag     int64  
 18  Game Date          int64  
 19  Home Team          object 
 20  Away Team          object 
 21  Season Type        object 
 22  height             float64
 23  weight             float64
 24  collage            object 
 25  born              

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10389755 entries, 0 to 10389754
Data columns (total 33 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   EVENTMSGACTIONTYPE         int64  
 1   EVENTMSGTYPE               int64  
 2   EVENTNUM                   int64  
 3   GAME_ID                    int64  
 4   HOMEDESCRIPTION            object 
 5   NEUTRALDESCRIPTION         float64
 6   PCTIMESTRING               object 
 7   PERIOD                     int64  
 8   PERSON1TYPE                float64
 9   PERSON2TYPE                int64  
 10  PERSON3TYPE                int64  
 11  PLAYER1_ID                 int64  
 12  PLAYER1_NAME               object 
 13  PLAYER1_TEAM_ABBREVIATION  object 
 14  PLAYER1_TEAM_CITY          object 
 15  PLAYER1_TEAM_ID            float64
 16  PLAYER1_TEAM_NICKNAME      object 
 17  PLAYER2_ID                 int64  
 18  PLAYER2_NAME               object 
 19  PLAYER2_TEAM_ABBREVIATION  object 
 20  

None

In [45]:
shot_data = shot_data.sort_values(by=['Game ID','Game Event ID'])

In [46]:
pbp_data.sort_values(by=['GAME_ID','EVENTNUM'],inplace=True)

In [47]:
pbp_data.head()

,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
418371,0,12,0,20000001,NaN,NaN,12:00,1,0.0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:13 PM
418372,0,10,1,20000001,Jump Ball Camby vs. Ratliff: Tip to Houston,NaN,12:00,1,4.0,5,...,275,Allan Houston,NYK,New York,1.610613e+09,Knicks,NaN,NaN,NaN,12:13 PM
418373,1,2,2,20000001,MISS Sprewell 6' Jump Shot,NaN,11:41,1,4.0,0,...,689,Theo Ratliff,PHI,Philadelphia,1.610613e+09,76ers,NaN,NaN,Ratliff BLOCK (1 BLK),12:14 PM
418374,0,4,3,20000001,NaN,NaN,11:40,1,3.0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76ers Rebound,12:14 PM
418375,2,6,4,20000001,Camby S.FOUL (P1.T1),NaN,11:29,1,4.0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:14 PM


In [48]:
shot_data.head()

,Game ID,Game Event ID,Player ID,Player Name,Team ID,Team Name,Period,Minutes Remaining,Seconds Remaining,Action Type,...,Game Date,Home Team,Away Team,Season Type,height,weight,collage,born,birth_city,birth_state
805985,20000001,2,84,Latrell Sprewell,1610612752,New York Knicks,1,11,41,Jump Shot,...,20001031,NYK,PHI,Regular Season,196.0,86.0,University of Alabama,1970.0,Milwaukee,Wisconsin
374699,20000001,13,275,Allan Houston,1610612752,New York Knicks,1,10,53,Jump Shot,...,20001031,NYK,PHI,Regular Season,198.0,90.0,University of Tennessee,1971.0,Louisville,Kentucky
334431,20000001,16,238,Tyrone Hill,1610612755,Philadelphia 76ers,1,10,35,Jump Shot,...,20001031,NYK,PHI,Regular Season,206.0,108.0,Xavier University,1968.0,Cincinnati,Ohio
805986,20000001,18,84,Latrell Sprewell,1610612752,New York Knicks,1,10,19,Jump Shot,...,20001031,NYK,PHI,Regular Season,196.0,86.0,University of Alabama,1970.0,Milwaukee,Wisconsin
632843,20000001,20,689,Theo Ratliff,1610612755,Philadelphia 76ers,1,10,8,Layup Shot,...,20001031,NYK,PHI,Regular Season,208.0,102.0,University of Wyoming,1973.0,Demopolis,Alabama


In [49]:
pbp_data.rename({'GAME_ID':'Game ID','EVENTNUM':'Game Event ID'},axis=1,inplace=True)

In [50]:
pbp_data.head()

,EVENTMSGACTIONTYPE,EVENTMSGTYPE,Game Event ID,Game ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
418371,0,12,0,20000001,NaN,NaN,12:00,1,0.0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:13 PM
418372,0,10,1,20000001,Jump Ball Camby vs. Ratliff: Tip to Houston,NaN,12:00,1,4.0,5,...,275,Allan Houston,NYK,New York,1.610613e+09,Knicks,NaN,NaN,NaN,12:13 PM
418373,1,2,2,20000001,MISS Sprewell 6' Jump Shot,NaN,11:41,1,4.0,0,...,689,Theo Ratliff,PHI,Philadelphia,1.610613e+09,76ers,NaN,NaN,Ratliff BLOCK (1 BLK),12:14 PM
418374,0,4,3,20000001,NaN,NaN,11:40,1,3.0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76ers Rebound,12:14 PM
418375,2,6,4,20000001,Camby S.FOUL (P1.T1),NaN,11:29,1,4.0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:14 PM


In [51]:
shot_data_merged = shot_data.merge(pbp_data,on=['Game ID','Game Event ID'],how='inner')

In [52]:
shot_data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3528098 entries, 0 to 3528097
Data columns (total 59 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Game ID                    int64  
 1   Game Event ID              int64  
 2   Player ID                  int64  
 3   Player Name                object 
 4   Team ID                    int64  
 5   Team Name                  object 
 6   Period                     int64  
 7   Minutes Remaining          int64  
 8   Seconds Remaining          int64  
 9   Action Type                object 
 10  Shot Type                  object 
 11  Shot Zone Basic            object 
 12  Shot Zone Area             object 
 13  Shot Zone Range            object 
 14  Shot Distance              int64  
 15  X Location                 int64  
 16  Y Location                 int64  
 17  Shot Made Flag             int64  
 18  Game Date                  int64  
 19  Home Team                  object 
 20  Aw

In [53]:
shot_data_merged.head()

,Game ID,Game Event ID,Player ID,Player Name,Team ID,Team Name,Period,Minutes Remaining,Seconds Remaining,Action Type,...,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING
0,20000001,2,84,Latrell Sprewell,1610612752,New York Knicks,1,11,41,Jump Shot,...,689,Theo Ratliff,PHI,Philadelphia,1.610613e+09,76ers,NaN,NaN,Ratliff BLOCK (1 BLK),12:14 PM
1,20000001,13,275,Allan Houston,1610612752,New York Knicks,1,10,53,Jump Shot,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:16 PM
2,20000001,16,238,Tyrone Hill,1610612755,Philadelphia 76ers,1,10,35,Jump Shot,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Hill 10' Jump Shot,12:17 PM
3,20000001,18,84,Latrell Sprewell,1610612752,New York Knicks,1,10,19,Jump Shot,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:18 PM
4,20000001,20,689,Theo Ratliff,1610612755,Philadelphia 76ers,1,10,8,Layup Shot,...,0,NaN,NaN,NaN,NaN,NaN,3 - 2,-1,Ratliff Layup (3 PTS) (Lynch 1 AST),12:18 PM


In [56]:
shot_data_merged.iloc[0,:]

Game ID                                        20000001
Game Event ID                                         2
Player ID                                            84
Player Name                            Latrell Sprewell
Team ID                                      1610612752
Team Name                               New York Knicks
Period                                                1
Minutes Remaining                                    11
Seconds Remaining                                    41
Action Type                                   Jump Shot
Shot Type                                2PT Field Goal
Shot Zone Basic                   In The Paint (Non-RA)
Shot Zone Area                                Center(C)
Shot Zone Range                         Less Than 8 ft.
Shot Distance                                         6
X Location                                          -58
Y Location                                           28
Shot Made Flag                                  

In [57]:
shot_data_merged.to_csv('../Data/shot_data_merged.csv')